In [115]:
from optparse import OptionParser
from ConfigUtils import *
from SocrataStuff import *
from Utils import *
from UpdateSocrataFields import *
import json
import pandas as pd
from pandas.io.json import json_normalize

In [116]:
class BuildDatasets:
    @staticmethod
    def getQryCols(tables, dataset_name):
        dataset = tables[dataset_name]
        dataset_cols = dataset['field_list']
        dataset_cols_qry = ",".join(dataset_cols)
        #if 'filter_qry' in dataset.keys():
        #    dataset_cols_qry = dataset_cols_qry + dataset['filter_qry']
        return dataset_cols_qry
            
    @staticmethod
    def makeDf(json_obj):
        df = json_normalize(json_obj)
        df = df.fillna('')
        return df

    @staticmethod
    def getDatasets(tables, socrataQueriesObject):
        dfs_dict = {}
        datasets = tables.keys()
        for dataset in datasets:
            fbf = tables[dataset]['fbf']
            qryCols = BuildDatasets.getQryCols(tables, dataset)
            results_json = socrataQueriesObject.pageThroughResultsSelect(fbf, qryCols)
            df = BuildDatasets.makeDf(results_json)
            #rename the fields
            if 'field_mapping' in tables[dataset].keys():
                df = PandasUtils.mapFieldNames(df, tables[dataset]['field_mapping'])
            dfs_dict[dataset] = df        
        return dfs_dict

In [117]:
fieldConfigFile = 'fieldConfigMasterDD.yaml'
config_inputdir = '/Users/j9/Desktop/fetch-socrata-fields/configs/'

cI =  ConfigItems(config_inputdir ,fieldConfigFile  )
configItems = cI.getConfigs()
sc = SocrataClient(config_inputdir, configItems)
client = sc.connectToSocrata()
clientItems = sc.connectToSocrataConfigItems()
lg = pyLogger(configItems)
logger = lg.setConfig()
socrataLoadUtils = SocrataLoadUtils(configItems)
scrud = SocrataCRUD(client, clientItems, configItems, logger)
sqry = SocrataQueries(clientItems, configItems)


tables = configItems['tables']
dfs_dict = BuildDatasets.getDatasets(tables, sqry)

In [118]:
df_sizes = {}
df_names = dfs_dict.keys()
for dataset in df_names:
    df_sizes[dataset] = len(dfs_dict[dataset])
print df_sizes

{'asset_fields': 7727, 'asset_inventory': 4978, 'data_dictionary_attachments': 167, 'global_fields': 193, 'data_dictionary_do_not_process': 8, 'cordinators': 66, 'dataset_inventory': 862, 'department_lookup': 56}


In [130]:
master_data_dictionary = configItems['master_data_dictionary']
fields_to_include = master_data_dictionary['fields_to_include']
base_url = configItems['base_url']

In [151]:
class MasterDataDictionary:
    
    @staticmethod
    def filter_base_datasets(dfs_dict):
        '''filter out irrelevant records in datasets'''
        #filter out the relevant data:
        asset_fields = dfs_dict['asset_fields']
        asset_inventory =  dfs_dict['asset_inventory']
        data_dictionary_attachments = dfs_dict['data_dictionary_attachments']
        dataset_inventory = dfs_dict['dataset_inventory']
        coordinators = dfs_dict['cordinators']  
        #asset_inventory-> we only want rows that are datsets
        asset_inventory = asset_inventory[asset_inventory['type']== 'dataset']
        #asset_inventory-> remove all the geo fields
        asset_fields = asset_fields[asset_fields['data_type'] ==  'tabular']
        print len(asset_fields)
        #filter out records that don't have data_dictionaries + remove dupes
        data_dictionary_attachments = data_dictionary_attachments[data_dictionary_attachments['data_dictionary_attached'] ==  True]
        data_dictionary_attachments = data_dictionary_attachments.drop_duplicates('datasetid')
        #grab the primary data coordinator
        coordinators = coordinators[coordinators['primary'] == 'Yes']
        return asset_fields, asset_inventory, data_dictionary_attachments, dataset_inventory, coordinators 
     
    @staticmethod
    def buildInventoryInfo(dataset_inventory,cordinators):
        '''joins the dataset inventory to the coordinators dataset'''
        dataset_inventory = pd.merge(dataset_inventory, cordinators, left_on='department_from_inventory', right_on='department', how='left')
        return dataset_inventory
    
    @staticmethod
    def build_base(dfs_dict):
        '''builds up all the non-transformed fields'''
        fields_to_include= ['columnid', 'datasetid', 'dataset_name', 'inventoryid', 'field_name', 'socrata_field_type', 'field_type', 'api_key', 'data_steward', 'data_steward_name', 'department_from_inventory', 'department_from_catalog', 'data_coordinator', 'data_dictionary_attached', 'attachment_url', 'field_description']
        asset_fields, asset_inventory, data_dictionary_attachments, dataset_inventory, coordinators = MasterDataDictionary.filter_base_datasets(dfs_dict)
        dataset_inventory =  MasterDataDictionary.buildInventoryInfo(dataset_inventory,coordinators)
        #join everything together to make the master dataset
        master_df = pd.merge(asset_fields, asset_inventory, on='datasetid', how='left').merge(data_dictionary_attachments, on='datasetid', how='left').merge(dataset_inventory, on='datasetid', how='left')
        print len(master_df)
        master_df = master_df.fillna('')
        master_df = master_df[fields_to_include ]
        return master_df
    
    
    @staticmethod
    def add_url(master_df, base_url):
        master_df['open_data_portal_url'] =  "http://"+ base_url+ '/resource/' + master_df['datasetid']
        return master_df
    
    @staticmethod
    def calcDepartmentField(master_df):
        df_master['department'] = 
    

    @staticmethod
    def addCalculatedFields(master_df, base_url):
        #fields_calculate= ['department', 'field_documented', 'global_field', 'do_not_process', 'field_definition', 'open_data_portal_url']
        master_df = MasterDataDictionary.add_url(master_df, base_url)
        return master_df
        
    
   

In [152]:
def groupbyCountStar(df, group_by_list):
    return df.groupby(group_by_list).size().reset_index(name='count')


In [156]:

# 	datasetid
mm_df = MasterDataDictionary.build_base(dfs_dict )
print len(mm_df)
mm_df = MasterDataDictionary.addCalculatedFields(mm_df, base_url)
#mm_df

#asset_data = dfs_dict['asset_fields']
#asset_data = asset_data[asset_data['data_type'] ==  'tabular']
#asset_data_geo = asset_data[asset_data['data_type'] ==  'geo']
#asset_data_geo

#column_id_cnts = groupbyCountStar(data_dictionary_attachments, ['datasetid'])
#column_id_cnts = column_id_cnts[column_id_cnts['count'] != 1]
#column_id_cnts

6908
6908
6908
